In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

ct_data = pd.read_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\DATA\CT\CT_SOLO.csv")
print(ct_data.info())

# Objetivos a conseguir con esta exploración inicial
# 1- Conocer la estructura y tipo de de datos con los que vamos a trabajar, además de un análisis descriptivo de los mismos.
# 2- Avergiuar que columnas no nos proporcionan información relevante y cuales si y reordenar el conjunto de datos.
# 3- Sacar información como distribución de valores nulos, outliers y todo lo típico de cara a la limpieza de datos.
# 4- Visualizar datos para sacar conclusiones preliminares.
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 500)
pd.set_option('display.show_dimensions', True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Columns: 384 entries, alt to videoViewCount
dtypes: bool(2), float64(158), int64(10), object(214)
memory usage: 80.8+ KB
None


In [18]:
from tabulate import tabulate
print(ct_data.shape)
head_df = ct_data.head(5)

print(tabulate(head_df, headers=head_df.columns, tablefmt='fancy_grid'))

(27, 384)
╒════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════╤══════════════════════════════╤═════════════════════════════════╤════════════════════════════════╤══════════════

In [19]:
#También vamos a eliminar la publicación SORTEO, pues es un tipo de publicación que no nos interesa analizar ya que independientemente de la hora a la que se publique va a tener una alta tasa de interacción.
ct_data_copy = ct_data.copy()

post_sorteo = ct_data_copy['caption'].str.contains('SORTEO')
#print(post_sorteo)
ct_data_copy = ct_data_copy[~post_sorteo]
print(ct_data_copy.shape)

#Vamos a ver los valores nulos que tenemos en el dataset y el porcentaje de valores nulos que hay en cada columna.
nulls = ct_data_copy.isnull().sum()
nulls = nulls[nulls > 0]
nulls = nulls/ct_data_copy.shape[0] 
print(nulls.sort_values(ascending=False))

(26, 384)
childPosts/7/ownerId                                1.000000
childPosts/3/timestamp                              1.000000
latestComments/7/id                                 1.000000
childPosts/6/firstComment                           1.000000
latestComments/6/timestamp                          1.000000
latestComments/6/text                               1.000000
latestComments/6/repliesCount                       1.000000
latestComments/6/ownerUsername                      1.000000
childPosts/6/caption                                1.000000
latestComments/6/ownerProfilePicUrl                 1.000000
childPosts/5/timestamp                              1.000000
childPosts/5/ownerId                                1.000000
childPosts/5/likesCount                             1.000000
childPosts/5/firstComment                           1.000000
childPosts/5/caption                                1.000000
latestComments/6/owner/username                     1.000000
childPosts/4/t

In [20]:
#Vamos a eliminar aquellas columnas que solo tienen valores nulos/vacíos/0 
all_nan_cols = [i for i in ct_data_copy if ct_data_copy[i].isnull().all()]
all_0_cols = [i for i in ct_data_copy if (ct_data_copy[i] == 0).all()]
all_vacio_cols = [i for i in ct_data_copy if (ct_data_copy[i] == '').all()]
print(all_nan_cols)
print(all_0_cols)
print(all_vacio_cols)
ct_data_copy = ct_data_copy.drop(all_nan_cols, axis=1)
ct_data_copy = ct_data_copy.drop(all_0_cols, axis=1)
ct_data_copy = ct_data_copy.drop(all_vacio_cols, axis=1)
print(ct_data_copy.shape)

['childPosts/0/caption', 'childPosts/0/firstComment', 'childPosts/0/likesCount', 'childPosts/0/ownerId', 'childPosts/0/timestamp', 'childPosts/1/caption', 'childPosts/1/firstComment', 'childPosts/1/likesCount', 'childPosts/1/ownerId', 'childPosts/1/timestamp', 'childPosts/2/caption', 'childPosts/2/firstComment', 'childPosts/2/likesCount', 'childPosts/2/ownerId', 'childPosts/2/timestamp', 'childPosts/3/caption', 'childPosts/3/firstComment', 'childPosts/3/likesCount', 'childPosts/3/ownerId', 'childPosts/3/timestamp', 'childPosts/4/caption', 'childPosts/4/firstComment', 'childPosts/4/likesCount', 'childPosts/4/ownerId', 'childPosts/4/timestamp', 'childPosts/5/caption', 'childPosts/5/firstComment', 'childPosts/5/likesCount', 'childPosts/5/ownerId', 'childPosts/5/timestamp', 'childPosts/6/caption', 'childPosts/6/firstComment', 'childPosts/6/likesCount', 'childPosts/6/ownerId', 'childPosts/6/timestamp', 'childPosts/7/caption', 'childPosts/7/firstComment', 'childPosts/7/likesCount', 'childPos

In [21]:
#Todas aquellas variables relacionadas con las dimensiones, fotos, url, shortcode, etc no nos interesan para el análisis de engagement, por lo que las eliminamos.
#Vamos a meter todas las palabras clave en una lista y eliminar las columnas que contengan esas palabras.
palabras_clave_para_eliminar = ['dimension', 'url', 'Url', 'shortCode', 'alt', 'id', 'Id', 'username', 'Username', 'text', 'firstComment', 'is_verified',' isPinned','latestComments' ]
columnas_a_eliminar = [col for col in ct_data_copy.columns if any(palabra in col for palabra in palabras_clave_para_eliminar)]
ct_data_copy = ct_data_copy.drop(columns=columnas_a_eliminar)
print(ct_data_copy.shape)
#for col in ct_data_copy.columns:
#    if 'dimension' in col or 'url' in col or 'Url' in col or 'shortCode' in col or 'alt' in col or 'id' in col or 'Id' in col or 'username' in col or 'Username' in col:
#        ct_data_copy = ct_data_copy.drop(col, axis=1)

#También vamos a eliminar las columnas que nos indican el tipo de cada post, ya que solo nos interesa saber el tipo que es la publicación, es decir, si es imagen, video o carrusel.
for i in range(0, 9):
        ct_data_copy = ct_data_copy.drop(f'childPosts/{i}/type', axis=1)
        ct_data_copy = ct_data_copy.drop(f'childPosts/{i}/commentsCount', axis=1)
ct_data_copy = ct_data_copy.drop('productType', axis=1)
print(ct_data_copy.shape)


(26, 49)
(26, 30)


In [22]:
#Esta celda ha sido usada para ver las columnas restantes conforme vamos eliminando las que no nos interesan. 
#Vamos a ver el porcentaje de cada columna de valores nulos tras haber elimminado algunas de las columnas que no nos interesan

nulls = ct_data_copy.isnull().sum()
nulls = nulls[nulls > 0]
nulls = nulls/ct_data_copy.shape[0] 
print(nulls.sort_values(ascending=False))

images/8                         0.961538
images/7                         0.923077
hashtags/9                       0.923077
images/6                         0.923077
isPinned                         0.884615
images/5                         0.884615
images/4                         0.807692
images/3                         0.769231
hashtags/8                       0.769231
images/2                         0.730769
musicInfo/song_name              0.730769
musicInfo/should_mute_audio      0.730769
musicInfo/artist_name            0.730769
musicInfo/uses_original_audio    0.730769
hashtags/7                       0.692308
hashtags/6                       0.653846
images/1                         0.576923
images/0                         0.576923
hashtags/5                       0.500000
hashtags/4                       0.461538
hashtags/1                       0.346154
hashtags/3                       0.346154
hashtags/2                       0.346154
hashtags/0                       0

In [23]:
#Tras haber revisado exhaustivamente las columnas y habernos deshecho de toda la información que no nos aportaba nada debido a todos los datos innecesarios que nos daba el scrapper,
#vamos a crear algunas columnas nuevas para algo más de información visual

#Comprobamos si la publicación tiene música o no, 1 = tiene música, 0 = no tiene música
ct_data_copy['hasMusic'] = ct_data_copy['musicInfo/song_name'].apply(lambda x: 0 if pd.isna(x) or x=='' else 1)
#head_df_2 = ct_data_copy.head(10)
#print(tabulate(head_df_2, headers=head_df_2.columns, tablefmt='fancy_grid'))

#Eliminamos toda otra columna en la que se hable de música, ya que no nos interesa para el análisis de engagement.
columns_to_drop = [col for col in ct_data_copy.columns if col.startswith('musicInfo/')]
ct_data_copy = ct_data_copy.drop(columns=columns_to_drop)
print(ct_data_copy.shape)

(26, 27)


In [24]:
#Vamos a ver el número de hastags que tiene cada publicación, como tenemos una columna del 1 al n número de variables indicando el texto de cada hastag, vamos a comprobar si cada hastag tiene texto 
# y si es así contamos 1 y lo vamos sumando
def count_hashtags(row):
    total = 0
    for i in range(10):
        col = f'hashtags/{i}'
        # Evitar error si no existe la columna
        if col in row:
            val = row[col]
            # condición: si no es nulo y no es una string vacía
            if pd.notna(val) and str(val).strip() != '':
                total += 1
    return total

ct_data_copy['num_hashtags'] = ct_data_copy.apply(count_hashtags, axis=1)

#print(ct_data_copy.head(10))


#Hacemos lo mismo para el número de imágenes que tiene cada publicación, como tenemos 3 tipos de publicación: imagen, video y sidecar, vamos a 
# declarar que 0 = video, 1 = imagen y i = sidecar siendo i= al número de imágenes que tiene la publicación.
def count_images(row):
    if row['type'] == 'Video':
        return 0
    
    if row['type'] == 'Image':
        return 1
    if row['type'] == 'Sidecar':
        total = 0
        for i in range(10):
            col = f'images/{i}'
            # Evitar error si no existe la columna
            if col in row:
                val = row[col]
                # condición: si no es nulo y no es una string vacía
                if pd.notna(val) and str(val).strip() != '':
                    total += 1
        return total
    return 0

ct_data_copy['num_images'] = ct_data_copy.apply(count_images, axis=1)
for i in range(0, 9):
        ct_data_copy = ct_data_copy.drop(f'images/{i}', axis=1)
print(tabulate(ct_data_copy.head(10), headers=ct_data_copy.columns, tablefmt='fancy_grid'))

╒════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════╤══════════════╤══════════════════╤════════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤════════════╤══════════════╤═══════════════════════════════════╤══════════════════════════╤═════════╤════════════╤════════════════╤══════════════╕
│    │ caption                                                                                                                                                                                                                                                                                

In [25]:
#Ahora vamos a obtener todos los hashtags y guardarlos en una lissta, luego vamos a ver las veces que han sido usados y la media de likes que tienen
# de cara a ver si el uso de alguno de estos supone una gran mejora en el engagement.
columns_hashtags = [col for col in ct_data_copy.columns if col.startswith('hashtags/')]
def collect_hashtags(row):
    tags = []
    for col in columns_hashtags:
        val = row[col]
        if pd.notna(val) and str(val).strip() != '':
            tags.append(str(val).strip().lower())  # normalizamos a minúsculas
    return tags

ct_data_copy['hashtags_list'] = ct_data_copy.apply(collect_hashtags, axis=1)
print(ct_data_copy['hashtags_list'])

0     [libros, lectora, lectura, acotar, alasdesangr...
1     [velas, libros, espiritualidad, cozy, fantasía...
2     [libros, acotar, rhysand, feyre, marcapaginas,...
3     [libros, binding13, chloewalsh, artesanía, hec...
4     [binding13, libros, marcapaginas, handmade, le...
5                                                    []
6                                                    []
7                                                    []
8     [velas, libros, hechoamano, sanvalentin, regal...
9     [sanvalentin, amorporloslibros, regalos, libro...
10                                                   []
11    [fundasliterarias, libros, bookstagram, lectur...
12                                                   []
13                                                   []
15    [libros, powerless, lectura, regalos, hechoama...
16                                                   []
17    [libros, artesanía, personalizado, elarchivode...
18    [lectorescozy, amorporloslibros, fantasíay

In [26]:
ct_exploded = ct_data_copy.explode('hashtags_list').reset_index(drop=True)

# RENOMBRAR la columna `hashtags_list` por `hashtag`
ct_exploded = ct_exploded.rename(columns={'hashtags_list': 'hashtag'})

stats_hashtags = (
    ct_exploded
    .groupby('hashtag')['likesCount']
    .agg(['count', 'mean', 'max'])
    .sort_values('count', ascending=False)
)

print(stats_hashtags.head(20))
stats_hashtags.to_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\DATA\CT\CT_SOLO_HASHTAGS_INFO.csv")

                  count       mean  max
hashtag                                
libros               17  36.588235   71
lectura              12  37.166667   71
artesanía             9  40.222222   71
hechoamano            8  39.750000   71
marcapaginas          7  43.285714   71
cozy                  7  27.000000   33
regalos               6  28.666667   44
madera                4  42.500000   71
acotar                3  58.000000   71
bookstagram           3  31.333333   33
velas                 2  33.000000   33
llaveros              2  51.500000   52
handmade              2  46.500000   51
original              2  44.500000   51
rhysand               2  61.500000   71
binding13             2  30.500000   42
amorporloslibros      2  40.000000   52
sanvalentin           2  30.500000   33
lecturacozy           2  35.500000   38
navidad               1  22.000000   22

[20 rows x 3 columns]


In [27]:
#Eliminamos otras columnas más
for i in range(0, 10):
        ct_data_copy = ct_data_copy.drop(f'hashtags/{i}', axis=1)
ct_data_copy = ct_data_copy.drop('isPinned', axis=1)
ct_data_copy = ct_data_copy.drop('ownerFullName', axis=1)
print(tabulate(ct_data_copy.head(10), headers=ct_data_copy.columns, tablefmt='fancy_grid'))

╒════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════╤══════════════╤══════════════════════════╤═════════╤════════════╤════════════════╤══════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│    │ caption                                                                                                                                                                                                                                                                                                                                                   

In [28]:
#Vamos a separar la fecha en que día de la semana fue y la hora de la publicación
ct_data_copy['timestamp'] = pd.to_datetime(ct_data_copy['timestamp'])

dias_semana = {
    0: 'Lunes',
    1: 'Martes',
    2: 'Miércoles',
    3: 'Jueves',
    4: 'Viernes',
    5: 'Sábado',
    6: 'Domingo'
}
#Mediante .dayofweek obtenemos el día de la semana en formato numérico, por lo que vamos a mapear esos números a los días de la semana
ct_data_copy['dia_semana'] = ct_data_copy['timestamp'].dt.dayofweek.map(dias_semana)

#Separamos la hora de la publicación
ct_data_copy['hora'] = ct_data_copy['timestamp'].dt.strftime('%H:%M:%S')
print(ct_data_copy[['timestamp', 'dia_semana', 'hora']].head(26))
ct_data_copy = ct_data_copy.drop('timestamp', axis=1)

                   timestamp dia_semana      hora
0  2024-12-11 09:31:56+00:00  Miércoles  09:31:56
1  2025-02-27 08:36:29+00:00     Jueves  08:36:29
2  2024-12-12 19:47:30+00:00     Jueves  19:47:30
3  2025-02-23 14:46:01+00:00    Domingo  14:46:01
4  2025-02-20 09:33:38+00:00     Jueves  09:33:38
5  2025-02-19 13:01:52+00:00  Miércoles  13:01:52
6  2025-02-18 09:51:02+00:00     Martes  09:51:02
7  2025-02-06 09:26:29+00:00     Jueves  09:26:29
8  2025-02-13 09:36:55+00:00     Jueves  09:36:55
9  2025-02-11 14:07:46+00:00     Martes  14:07:46
10 2025-02-25 09:13:08+00:00     Martes  09:13:08
11 2025-02-12 09:00:32+00:00  Miércoles  09:00:32
12 2025-01-15 13:39:02+00:00  Miércoles  13:39:02
13 2024-12-30 18:47:31+00:00      Lunes  18:47:31
15 2024-12-24 11:37:32+00:00     Martes  11:37:32
16 2024-12-29 21:24:52+00:00    Domingo  21:24:52
17 2025-01-09 11:07:09+00:00     Jueves  11:07:09
18 2025-01-23 08:09:49+00:00     Jueves  08:09:49
19 2024-12-21 12:00:30+00:00     Sábado  12:00:30


In [29]:
#Creamos la columna id para poder tener un índice de cada publicación.
ct_data_copy['id'] = ct_data_copy.index
print(tabulate(ct_data_copy.head(10), headers=ct_data_copy.columns, tablefmt='fancy_grid'))

╒════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════╤══════════════╤═════════╤════════════╤════════════════╤══════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════╤══════════╤══════╕
│    │ caption                                                                                                                                                                                                                                                                                                                                             

In [31]:
#Ya hemos realizado un formateo del csv y nos hemos quedado con lo más importante, aunque posiblemente añadamos más variables relacionadas con estadísticas en un futuro. 
# Por último toca clasificar que tipo de publicación es cada post, pues nos interesa saber que producto o tipo de publicación iene más engagement y así aumentar el foco en este para aumentar las ventas.

#La idea inicial entrenar un modelo de clasificación para predecir el tipo de publicación, pero no tenemos la suficiente cantidad de datos para hacerlo. 
#Dado que tenemos pocos posts podriamos hacerlo a mano, pero esto no es una buena práctica pues la idea es que este proyecto pueda ser aplicado a cualquier cuenta sin importar su cantidad de publicaciones.
#Para ellos vamos a hacer uso de una api open source de un LLM que mediante un prompt se encarge de leer la descripción de la publicación y clasificarla.

#Vamos a sacr el csv del id de la publicación y su descripción para poder clasificarlo con el modelo de clasificación.
ct_data_copy_prompt = ct_data_copy[['id', 'caption']]
ct_data_copy_prompt.to_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\DATA\CT\CT_SOLO_PROMPT.csv", index=False)

In [15]:
output = pd.read_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\output\clasificacion_publicaciones.csv")
print(output[['id', 'tipo_de_publicacion', 'tipo_de_producto']])

                                                   id  \
0                                                   0   
1                                                   1   
2                                                   2   
3                                                   3   
4                                                   4   
5                                                   5   
6                                                   6   
7                                                   7   
8                                                   8   
9                                                   9   
10                                                 10   
11                                                 11   
12  Si eres de las que no pueden ir a ningún lado ...   
13   Hoy te traigo estos modelos de fundas acolchadas   
14  🌸 Funda de flores rosa – ¡Última unidad! Si te...   
15  💙 Funda de flores azules – Un diseño elegante ...   
16  🤍 Funda de pelito beige sin